In [1]:
import pandas as pd
import numpy as np
from datetime import datetime
from datetime import timedelta

import warnings
warnings.filterwarnings("ignore")

##### step 1: 讀進 IJL1-7的 op 卡檔
##### step 2: 形成 pi 點位路徑
##### step 3: 形成 英中對照字典
##### step 4: 形成 各機台字典, key:設備編號，value: 中文清單列表、英文清單列表

In [16]:
def NameDict(df, k , v):
    d = dict([(a, b) for a, b in zip(df[k], df[v])])
    return d

In [17]:
def MachineDict(op):
    """
    輸入: op資料
    輸出: 字典，keys: (str)設備編號，values: (list) 中文點位list(第0元素) 跟 英文點位 list
    """
    dict_part = dict()
    for part in set(op['設備編號(ERP)'].values):
        T_ch = op[op['設備編號(ERP)']==part]['Ch_Name'].to_list()
        T_en = op[op['設備編號(ERP)']==part]['Tag_Name'].to_list()
        T = []
        T.append(T_ch)
        T.append(T_en)
        dict_part[part]= T

    return dict_part

In [18]:
# step 1: 讀進 IJL1-7的 op 卡檔
# step 2: 形成 pi 點位路徑
# step 3: 建立英中轉換的 dict , key 英文點位, value 中文點位 / 之後以中文來做篩選
# step 4: 建立設備編號對應的字典，values 是中英文點位
pisource = 'pi:\\10.114.134.1\\'

allop = pd.ExcelFile('嘉一廠_OP_射出.xlsx')
sheet_names = allop.sheet_names

In [19]:
for i in range(len(sheet_names)):
    tmp_name = sheet_names[i]
    tmp_df = allop.parse(tmp_name)
    globals()[tmp_name + '_op'] = tmp_df
    globals()[tmp_name + '_pitag'] = [pisource + elm for elm in list(tmp_df['Tag_Name'])]
    globals()[tmp_name + '_namedict'] = NameDict(tmp_df, 'Tag_Name', 'Ch_Name')
    globals()[tmp_name + '_machinedict'] = MachineDict(tmp_df)


In [20]:
def TagExtract(op):

    # 全部點位 list
    all_tag = list(op['Ch_Name'].values)

    # IJL1-7 有啟用值 的點位只有一個, 就是生產狀態，此部分相對好判斷，但適用性不高
    # 看嘉一射出 excel 其他的只要啟用值是T, 都是生產狀態
    # 用簡單判斷
    enable_tag = list(op[op['有無啟用值']=='T']['Ch_Name'].values)

    # 判斷 無啟用值, 有設定值 的關鍵字 list

    temp = list(op[((op['有無啟用值']=='F') & (op['有無設定值']=='T'))]['Ch_Name'].values)
    set_tag = [item.split('_SV')[0] for item in temp if '_SV' in item]
    
    return all_tag, enable_tag, set_tag    

In [21]:
a,b,c = TagExtract(IJN8_op)

### chapter 2 : 抓資料

In [38]:
from datascratch import getPIParameters, PILogin, PICatchParametersData, data_export
from PIWebApiQuery0914_copy import QueryData, Get, GetLast, PILogin, data_export1

In [277]:
pd.set_option('display.max_columns', 50)
pd.set_option('display.max_rows', 5)

In [43]:
time_format="%y-%m-%d %H:%M:%S"
df = data_export(datetime.strftime(datetime.now()-timedelta(minutes=15), time_format),
                datetime.strftime(datetime.now(), time_format),
                tagpoint_list=IJN9_pitag, time_interval='1m')

In [44]:
df

,INJ_CC_IJN9_MESProductStatus,INJ_CC_IJN9_H1_PV,INJ_CC_IJN9_H1_SV,INJ_CC_IJN9_H2_PV,INJ_CC_IJN9_H2_SV,INJ_CC_IJN9_H3_PV,INJ_CC_IJN9_H3_SV,INJ_CC_IJN9_H4_PV,INJ_CC_IJN9_H4_SV,INJ_CC_IJN9_H5_PV,INJ_CC_IJN9_H5_SV,INJ_CC_IJN9_H6_PV,INJ_CC_IJN9_H6_SV,INJ_CC_IJN9_HN_PV,INJ_CC_IJN9_HN_SV
Timestamp,,,,,,,,,,,,,,,
2023-02-04 09:45:48,1.0,151.9,140.0,155.3,155.0,155.8,155.0,155.4,155.0,155.5,155.0,155.2,155.0,140.4,140.0
2023-02-04 09:46:48,1.0,151.9,140.0,155.3,155.0,155.8,155.0,155.4,155.0,155.5,155.0,155.2,155.0,140.4,140.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-02-04 09:59:48,1.0,151.9,140.0,155.3,155.0,155.8,155.0,155.4,155.0,155.5,155.0,155.2,155.0,140.4,140.0
2023-02-04 10:00:48,1.0,151.9,140.0,155.3,155.0,155.8,155.0,155.4,155.0,155.5,155.0,155.2,155.0,140.4,140.0


In [29]:
ch_df = df.rename(columns=IJN7_namedict)

In [34]:
f,g,h,j = HealthValue(ch_df, IJN7_op)

In [33]:
# 計算分數跟看是否正常生產寫在同一個func

# 現在有點位清單, 計算分數
# df 是中文column的資料, op 是 op卡
def HealthValue(df, op):
    # 這邊是中文點為, set_tag 是 key word
    all_tag, enable_tag, set_tag = TagExtract(op)
    df_res = pd.DataFrame(index = df.index, columns = set_tag)
    df_final = pd.DataFrame(index = ['residual', 'f_i1', 'F_i'], columns = set_tag)

    if (df[enable_tag[0]].dtypes == object) or (df[enable_tag[0]].mean()==0.0):
        work_state = '非正常生產狀態'
        df_res[df_res != 0] = 0
        df_final[df_final != 0] = 0
        score = 0
    
    elif (df[enable_tag[0]].dtypes != object) and (df[enable_tag[0]].mean()==1.0):
        work_state = '正常生產狀態'

        for i, col in enumerate(set_tag):
            usecol = [ item for item in all_tag if col in item]
            if (df[usecol[0]].dtypes !=object) and (df[usecol[1]].dtypes != object):
                df_res[col] = abs(df[usecol[0]]-df[usecol[1]])
            else:
                df_res[col] = np.zeros(len(df[usecol[0]]))
            limit = op[op['Ch_Name']==usecol[0]]['允差值'].values[0]
            meannow = df_res[col].mean()
            
            # 個別扣分跟點位分數
            f_i1 = 20*(meannow/limit)
            temp = 100-f_i1
            if temp < 0:
                temp =0
            df_final[col]=[limit, f_i1, temp]

        # df.loc['..']是 series, df.loc[['..']] 是 dataframe
        if df_final.loc['F_i'].min() <80:
            score = df_final.loc['F_i'].min()
        elif df_final.loc['F_i'].min() >= 80:
            score = df_final.loc['F_i'].mean()

    return work_state, df_res, df_final, score
    



['IJL1_H1', 'IJL1_H1_SV']

In [169]:
a, b, c, d  = HealthValue(ch_df, IJL1_op)

In [195]:
k = globals()['IJL5'+'_pitag']


In [196]:
I

['pi:\\10.114.134.1\\INJ_CC_IJL5_MESProductStatus',
 'pi:\\10.114.134.1\\INJ_CC_IJL5_H1_PV',
 'pi:\\10.114.134.1\\INJ_CC_IJL5_H1_SV',
 'pi:\\10.114.134.1\\INJ_CC_IJL5_H2_PV',
 'pi:\\10.114.134.1\\INJ_CC_IJL5_H2_SV',
 'pi:\\10.114.134.1\\INJ_CC_IJL5_H3_PV',
 'pi:\\10.114.134.1\\INJ_CC_IJL5_H3_SV',
 'pi:\\10.114.134.1\\INJ_CC_IJL5_H4_PV',
 'pi:\\10.114.134.1\\INJ_CC_IJL5_H4_SV',
 'pi:\\10.114.134.1\\INJ_CC_IJL5_HN_PV',
 'pi:\\10.114.134.1\\INJ_CC_IJL5_HN_SV',
 'pi:\\10.114.134.1\\INJ_CC_IJL5_HEN_PV',
 'pi:\\10.114.134.1\\INJ_CC_IJL5_HEN_SV']

### chapter 2: 修改 棧板 op 卡資料

In [279]:
nj_allop = pd.ExcelFile('嘉一廠_OP_棧板6.xlsx')
nj_sheet_names = nj_allop.sheet_names

In [280]:
nj_sheet_names

['NJ01', 'NJ02', 'NJ03', 'NJ04', 'NJ05', 'NJ06', 'CJ01', 'CJ02']

In [281]:
for sheet in nj_sheet_names:
    tmp_df = nj_allop.parse(sheet)
    tmp_list = list(tmp_df['Ch_Name'].values)
    if sheet in ['NJ05', 'NJ06']:
        for i, col in enumerate(tmp_list):
            if '生產狀態' in col:
                continue
            elif '設定_':
                tmp_list[i] = tmp_list[i].replace('設定_','')
    elif sheet in ['CJ01','CJ02']:
        for i, col in enumerate(tmp_list):
            if '生產狀態' in col:
                continue
            else:
                reg = re.search('[0-9]',tmp_list[i])
                tmp_list[i] = tmp_list[i].replace(reg.group(),'').replace('溫度', '溫度'+reg.group())
    else:
        for i, col in enumerate(tmp_list):
            if '生產狀態' in col:
                continue
            else:
                reg = re.search('[0-9a-zA-Z]+', tmp_list[i])
                tmp_list[i] = tmp_list[i].replace(reg.group(), '').replace('_', reg.group())
        
    tmp_df['Ch_Name'] = tmp_list
    globals()[sheet+'_op'] = tmp_df
    globals()[sheet + '_pitag'] = [pisource + elm for elm in list(tmp_df['Tag_Name'])]
    globals()[sheet + '_namedict'] = NameDict(tmp_df, 'Tag_Name', 'Ch_Name')
    globals()[sheet + '_machinedict'] = MachineDict(tmp_df)

###  抓資料測試棧板

In [192]:
time_format="%y-%m-%d %H:%M:%S"
df = data_export(datetime.strftime(datetime.now()-timedelta(minutes=15), time_format),
                datetime.strftime(datetime.now(), time_format),
                tagpoint_list=CJ01_pitag, time_interval='1m')

In [194]:
chdf = df.rename(columns = CJ01_namedict)

In [203]:
all_tag, enable_tag, set_tag = TagExtract(CJ01_op)

In [209]:
# tagextract 有問題
set_tag

[]

In [212]:
CJ01_op['設備編號(ERP)'].values[0]

'CJ01'

In [236]:
def TagExtract(op):

    # 全部點位 list
    all_tag = list(op['Ch_Name'].values)

    # IJL1-7 有啟用值 的點位只有一個, 就是生產狀態，此部分相對好判斷，但適用性不高
    # 看嘉一射出 excel 其他的只要啟用值是T, 都是生產狀態
    # 用簡單判斷
    enable_tag = list(op[op['有無啟用值']=='T']['Ch_Name'].values)

    # 判斷 無啟用值, 有設定值 的關鍵字 list

    temp = list(op[((op['有無啟用值']=='F') & (op['有無設定值']=='T'))]['Ch_Name'].values)

    nj_sheet_names = ['NJ01', 'NJ02', 'NJ03', 'NJ04', 'NJ05', 'NJ06', 'CJ01', 'CJ02']

    if op['設備編號(ERP)'].values[0] in nj_sheet_names:
        set_tag = [item.split('設定')[0] for item in temp if '設定' in item]
    else:
        set_tag = [item.split('_SV')[0] for item in temp if '_SV' in item]
    
    return all_tag, enable_tag, set_tag

In [291]:
time_format="%y-%m-%d %H:%M:%S"
df = data_export(datetime.strftime(datetime.now()-timedelta(minutes=15), time_format),
                datetime.strftime(datetime.now(), time_format),
                tagpoint_list=NJ03_pitag, time_interval='1m')

In [293]:
chdf = df.rename(columns = NJ03_namedict)

In [294]:
chdf

,NJ03_生產狀態,套桶加熱區1設定值,套桶加熱區2設定值,套桶加熱區3設定值,套桶加熱區4設定值,套桶加熱區5設定值,套桶加熱區6設定值,套桶加熱區7設定值,套桶加熱區8設定值,套桶加熱區9設定值,套桶加熱區1實際值,套桶加熱區2實際值,套桶加熱區3實際值,套桶加熱區4實際值,套桶加熱區5實際值,套桶加熱區6實際值,套桶加熱區7實際值,套桶加熱區8實際值,套桶加熱區9實際值
Timestamp,,,,,,,,,,,,,,,,,,,
2023-02-06 15:03:45,1.0,235.0,265.0,265.0,265.0,265.0,265.0,265.0,225.0,0.0,235.0,265.0,266.0,265.0,265.0,265.0,265.0,224.0,0.0
2023-02-06 15:04:45,1.0,235.0,265.0,265.0,265.0,265.0,265.0,265.0,225.0,0.0,235.0,265.0,265.0,265.0,265.0,265.0,265.0,224.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-02-06 15:17:45,1.0,235.0,265.0,265.0,265.0,265.0,265.0,265.0,225.0,0.0,235.0,265.0,266.0,265.0,265.0,265.0,265.0,225.0,0.0
2023-02-06 15:18:45,1.0,235.0,265.0,265.0,265.0,265.0,265.0,265.0,225.0,0.0,235.0,266.0,266.0,265.0,266.0,265.0,265.0,225.0,0.0


In [295]:
all_tag, enable_tag, set_tag = TagExtract(NJ03_op)

In [300]:
df_res = pd.DataFrame(index = chdf.index, columns = set_tag)
df_final = pd.DataFrame(index = ['residual', 'f_i1', 'F_i'], columns = set_tag)

In [301]:
df_final

,套桶加熱區1,套桶加熱區2,套桶加熱區3,套桶加熱區4,套桶加熱區5,套桶加熱區6,套桶加熱區7,套桶加熱區8,套桶加熱區9
residual,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
f_i1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
F_i,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [302]:
if (chdf[enable_tag[0]].dtypes == object) or (chdf[enable_tag[0]].mean()==0.0):
    work_state = '非正常生產狀態'
    df_res[df_res != 0] = 0
    df_final[df_final != 0] = 0
    score = 0
else:
    print('good')

good


In [312]:
pd.set_option('display.max_rows',50)

In [313]:
chdf

,NJ03_生產狀態,套桶加熱區1設定值,套桶加熱區2設定值,套桶加熱區3設定值,套桶加熱區4設定值,套桶加熱區5設定值,套桶加熱區6設定值,套桶加熱區7設定值,套桶加熱區8設定值,套桶加熱區9設定值,套桶加熱區1實際值,套桶加熱區2實際值,套桶加熱區3實際值,套桶加熱區4實際值,套桶加熱區5實際值,套桶加熱區6實際值,套桶加熱區7實際值,套桶加熱區8實際值,套桶加熱區9實際值
Timestamp,,,,,,,,,,,,,,,,,,,
2023-02-06 15:03:45,1.0,235.0,265.0,265.0,265.0,265.0,265.0,265.0,225.0,0.0,235.0,265.0,266.0,265.0,265.0,265.0,265.0,224.0,0.0
2023-02-06 15:04:45,1.0,235.0,265.0,265.0,265.0,265.0,265.0,265.0,225.0,0.0,235.0,265.0,265.0,265.0,265.0,265.0,265.0,224.0,0.0
2023-02-06 15:05:45,1.0,235.0,265.0,265.0,265.0,265.0,265.0,265.0,225.0,0.0,235.0,265.0,265.0,265.0,265.0,265.0,265.0,224.0,0.0
2023-02-06 15:06:45,1.0,235.0,265.0,265.0,265.0,265.0,265.0,265.0,225.0,0.0,235.0,265.0,266.0,265.0,265.0,265.0,265.0,225.0,0.0
2023-02-06 15:07:45,1.0,235.0,265.0,265.0,265.0,265.0,265.0,265.0,225.0,0.0,235.0,265.0,266.0,266.0,265.0,265.0,265.0,225.0,0.0
2023-02-06 15:08:45,1.0,235.0,265.0,265.0,265.0,265.0,265.0,265.0,225.0,0.0,235.0,265.0,266.0,265.0,265.0,265.0,266.0,225.0,0.0
2023-02-06 15:09:45,1.0,235.0,265.0,265.0,265.0,265.0,265.0,265.0,225.0,0.0,235.0,266.0,266.0,265.0,265.0,265.0,265.0,225.0,0.0
2023-02-06 15:10:45,1.0,235.0,265.0,265.0,265.0,265.0,265.0,265.0,225.0,0.0,235.0,266.0,266.0,265.0,266.0,265.0,266.0,226.0,0.0
2023-02-06 15:11:45,1.0,235.0,265.0,265.0,265.0,265.0,265.0,265.0,225.0,0.0,235.0,266.0,266.0,265.0,266.0,265.0,265.0,225.0,0.0


In [310]:
list(chdf[enable_tag[0]])

[1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 0.0,
 0.0,
 1.0,
 1.0,
 1.0,
 1.0]

In [303]:
if (chdf[enable_tag[0]].dtypes != object) and (chdf[enable_tag[0]].mean()==1.0):
    work_state = '正常生產狀態'
    print(work_state)

In [258]:
for i, col in enumerate(set_tag):
    usecol = [ item for item in all_tag if col in item]
    print(usecol)
    if (chdf[usecol[0]].dtypes !=object) and (chdf[usecol[1]].dtypes != object):
        df_res[col] = abs(chdf[usecol[0]]-chdf[usecol[1]])
    else:
        df_res[col] = np.zeros(len(chdf[usecol[0]]))
    limit = NJ02_op[NJ02_op['Ch_Name']==usecol[0]]['允差值'].values[0]
    meannow = df_res[col].mean()
    
    # 個別扣分跟點位分數
    f_i1 = 20*(meannow/limit)
    temp = 100-f_i1
    if temp < 0:
        temp =0
    df_final[col]=[limit, f_i1, temp]

# df.loc['..']是 series, df.loc[['..']] 是 dataframe
if df_final.loc['F_i'].min() <80:
    score = df_final.loc['F_i'].min()
elif df_final.loc['F_i'].min() >= 80:
    score = df_final.loc['F_i'].mean()

['套筒加熱區0_設定值', '套筒加熱區0_實際值']
['套筒加熱區1_設定值']


IndexError: list index out of range

In [262]:
usecol1 = [ item for item in all_tag if '套筒加熱區1_' in item]

In [263]:
print(usecol1)

['套筒加熱區1_設定值']


In [260]:
all_tag

['NJ02_生產狀態',
 '套筒加熱區0_設定值',
 '套筒加熱區1_設定值',
 '套筒加熱區2_設定值',
 '套筒加熱區3_設定值',
 '套筒加熱區4_設定值',
 '套筒加熱區5_設定值',
 '套筒加熱區6_設定值',
 '套筒加熱區7_設定值',
 '套筒加熱區8_設定值',
 '套筒加熱區9A_設定值',
 '套筒加熱區9B_設定值',
 '套筒加熱區9C_設定值',
 '套筒加熱區10A_設定值',
 '套筒加熱區10B_設定值',
 '套筒加熱區11_設定值',
 '套筒加熱區12_設定值',
 '套筒加熱區13_設定值',
 '套筒加熱區14_設定值',
 '套筒加熱區15_設定值',
 '套筒加熱區0_實際值',
 '套桶加熱區1_實際值',
 '套桶加熱區2_實際值',
 '套桶加熱區3_實際值',
 '套桶加熱區4_實際值',
 '套桶加熱區5_實際值',
 '套桶加熱區6_實際值',
 '套桶加熱區7_實際值',
 '套桶加熱區8_實際值',
 '套筒加熱區9A_實際值',
 '套筒加熱區9B_實際值',
 '套筒加熱區9C_實際值',
 '套筒加熱區10A_實際值',
 '套筒加熱區10B_實際值',
 '套筒加熱區11_實際值',
 '套筒加熱區12_實際值',
 '套筒加熱區13_實際值',
 '套筒加熱區14_實際值',
 '套筒加熱區15_實際值']

In [ ]:
my

In [231]:
b

['NJ02_生產狀態']

In [232]:
c

['套筒加熱區0_',
 '套筒加熱區1_',
 '套筒加熱區2_',
 '套筒加熱區3_',
 '套筒加熱區4_',
 '套筒加熱區5_',
 '套筒加熱區6_',
 '套筒加熱區7_',
 '套筒加熱區8_',
 '套筒加熱區9A_',
 '套筒加熱區9B_',
 '套筒加熱區9C_',
 '套筒加熱區10A_',
 '套筒加熱區10B_',
 '套筒加熱區11_',
 '套筒加熱區12_',
 '套筒加熱區13_',
 '套筒加熱區14_',
 '套筒加熱區15_']

In [ ]:
chdf = df.rename(columns = namedict)

state, df_res, df_final, final_score = HealthValue(chdf, op)

df_score = df_final.loc[['F_i'],:]
df_score.index = [datetime.now()]

df_err = ErrTable(df_score)
df_err = df_err.sort_values(by = 'tagscore', ascending = True)

In [218]:
nj_sheet_names

['NJ01', 'NJ02', 'NJ03', 'NJ04', 'NJ05', 'NJ06', 'CJ01', 'CJ02']

In [148]:
pd.set_option('display.max_rows',50)